In [7]:
# Capstone Exploratory Analysis Notebook
# This file is a Jupyter-ready Python script (cells separated by '# %%').
# It builds a full pipeline to:
# 1) Load IMU.csv
# 2) Run preprocessing (preprocessing.py)
# 3) Apply Kalman filter (kalman_filter.py)
# 4) Create sequence datasets
# 5) Train/evaluate multiple ML models (LSTM, Bi-LSTM, CNN+LSTM, Transformer, ResNet1D, XGBoost, RF, SVR)
# 6) Produce metrics, error analysis plots and trajectory comparisons

# Notes for use:
# - Place this file in the same environment where your IMU.csv, preprocessing.py and kalman_filter.py live (e.g. /mnt/data/).
# - At the top of the file are configuration options (seq_len, epochs, batch_size, etc.). Change as needed.
# - Required packages: numpy, pandas, scikit-learn, matplotlib, tensorflow, xgboost, joblib.
# - This file is intentionally verbose and explicit for reproducibility in a research setting.

# %%
# CONFIG
SEQ_LEN = 50             # number of timesteps used as input to sequence models
TEST_SIZE = 0.2
RANDOM_STATE = 42
EPOCHS = 50
BATCH_SIZE = 64
VERBOSE = 1

# %%
# IMPORTS
import os, sys, time
sys.path.append('/mnt/data')  # make sure your uploaded modules are importable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# local modules (preprocessing & kalman)
from preprocessing import preprocess_data
from kalman_filter import apply_kalman_filter

# deep learning
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout, BatchNormalization, Add, Activation, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention

# xgboost
import xgboost as xgb

# utilities
import math

# %%
# HELPERS: distance, metrics, sequence builders, plotting

def haversine_dist(lat1, lon1, lat2, lon2):
    # all args in degrees, returns meters
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c


def compute_trajectory_length_km(latitudes, longitudes):
    d = 0.0
    for i in range(1, len(latitudes)):
        d += haversine_dist(latitudes[i-1], longitudes[i-1], latitudes[i], longitudes[i])
    return d/1000.0


def compute_metrics_latlon(y_true, y_pred):
    # y_true, y_pred: arrays shape (n_samples, 2) in lat/lon degrees
    # convert to distance in meters per sample
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    rmse = np.sqrt(np.mean(dists**2))
    mae = np.mean(np.abs(dists))
    # R2 for lat and lon separately
    r2_lat = r2_score(y_true[:,0], y_pred[:,0])
    r2_lon = r2_score(y_true[:,1], y_pred[:,1])
    r2 = (r2_lat + r2_lon) / 2.0
    avg_distance_error = np.mean(dists)
    # compute trajectory length (based on true positions)
    traj_km = compute_trajectory_length_km(y_true[:,0], y_true[:,1])
    drift_per_km = (avg_distance_error / 1000.0) / (traj_km if traj_km>0 else np.nan)
    return {
        'rmse_m': rmse,
        'mae_m': mae,
        'r2': r2,
        'avg_distance_error_m': avg_distance_error,
        'drift_per_km_m_per_km': drift_per_km,
        'traj_km': traj_km
    }


def print_first_n_preds(y_true, y_pred, n=10):
    n = min(n, len(y_true))
    df = pd.DataFrame({
        'true_lat': y_true[:n,0],
        'true_lon': y_true[:n,1],
        'pred_lat': y_pred[:n,0],
        'pred_lon': y_pred[:n,1]
    })
    display(df)


def plot_trajectory(y_true, y_pred, title='Trajectory (true vs pred)', figsize=(8,6)):
    plt.figure(figsize=figsize)
    plt.plot(y_true[:,1], y_true[:,0], label='True', linewidth=2)
    plt.plot(y_pred[:,1], y_pred[:,0], label='Predicted', linewidth=1)
    plt.xlabel('Longitude'); plt.ylabel('Latitude')
    plt.title(title)
    plt.legend()
    plt.show()


def plot_error_analysis(y_true, y_pred):
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.hist(dists, bins=50)
    plt.title('Distance error histogram (m)')
    plt.subplot(1,3,2)
    plt.plot(dists)
    plt.title('Distance error over samples')
    plt.subplot(1,3,3)
    plt.scatter(y_true[:,0], dists, s=6)
    plt.title('Error vs Latitude')
    plt.tight_layout()
    plt.show()

# %%
# DATA LOADING + PREPROCESSING (this block will run first when you run the notebook)

def load_dataset(path='/mnt/data/IMU.csv'):
    df = pd.read_csv(path)
    # basic cleanup
    df = df.dropna(how='any').reset_index(drop=True)
    return df

# %%
# Sequence creation for sequence models

def create_sequences(X, y, seq_len=SEQ_LEN):
    """Return Xs shaped (n_samples, seq_len, n_features) and ys (n_samples, 2)
    using sliding window. We take sequences of sensors and predict the immediate next GPS point.
    """
    n_samples = X.shape[0] - seq_len
    if n_samples <= 0:
        raise ValueError('Dataset too small for given seq_len')
    Xs = np.zeros((n_samples, seq_len, X.shape[1]))
    ys = np.zeros((n_samples, 2))
    for i in range(n_samples):
        Xs[i] = X[i:i+seq_len]
        ys[i] = y[i+seq_len]  # predict the position right after the sequence
    return Xs, ys

# %%
# MODEL BUILDERS (Keras)

def build_lstm(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = LSTM(128, return_sequences=False)(inp)
    x = Dense(64, activation='relu')(x)
    out = Dense(2, activation='linear')(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model


def build_bilstm(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = Bidirectional(LSTM(96, return_sequences=False))(inp)
    x = Dense(64, activation='relu')(x)
    out = Dense(2, activation='linear')(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model


def build_cnn_lstm(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = Conv1D(64, 3, padding='same', activation='relu')(inp)
    x = MaxPooling1D(pool_size=2)(x)
    x = LSTM(64)(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(2)(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model


def build_transformer(seq_len, n_feats, head_size=64, num_heads=4, ff_dim=128):
    # Very small transformer encoder wrapper
    inp = Input(shape=(seq_len, n_feats))
    x = Dense(head_size)(inp)
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    x = LayerNormalization()(attn + x)
    # feed-forward
    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(head_size)(ff)
    x = LayerNormalization()(ff + x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(2)(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model


def build_resnet1d(seq_len, n_feats):
    def residual_block(x, filters, kernel_size=3):
        y = Conv1D(filters, kernel_size, padding='same')(x)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv1D(filters, kernel_size, padding='same')(y)
        y = BatchNormalization()(y)
        y = Add()([x, y])
        y = Activation('relu')(y)
        return y

    inp = Input(shape=(seq_len, n_feats))
    x = Conv1D(64, 3, padding='same')(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(2)(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

# %%
# TRAIN / EVALUATION LOOP (high level function)

def train_and_evaluate_keras(model_builder, X_train, y_train, X_test, y_test, name='model'):
    model = model_builder(X_train.shape[1], X_train.shape[2]) if callable(model_builder) else model_builder
    es = EarlyStopping(patience=6, restore_best_weights=True)
    t0 = time.perf_counter()
    hist = model.fit(X_train, y_train, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[es], verbose=VERBOSE)
    training_time = time.perf_counter() - t0
    # inference time
    t1 = time.perf_counter()
    y_pred = model.predict(X_test)
    inference_time = time.perf_counter() - t1
    metrics = compute_metrics_latlon(y_test, y_pred)
    metrics.update({'inference_time_s': inference_time, 'training_time_s': training_time})
    return model, y_pred, metrics, hist

# For sklearn/xgboost models (expect 2D input): wrap with MultiOutputRegressor

def train_and_evaluate_regressor(regressor, X_train, y_train, X_test, y_test, name='reg'):
    t0 = time.perf_counter()
    reg = MultiOutputRegressor(regressor)
    reg.fit(X_train, y_train)
    training_time = time.perf_counter() - t0
    t1 = time.perf_counter()
    y_pred = reg.predict(X_test)
    inference_time = time.perf_counter() - t1
    metrics = compute_metrics_latlon(y_test, y_pred)
    metrics.update({'inference_time_s': inference_time, 'training_time_s': training_time})
    return reg, y_pred, metrics

# %%
# This file continues with orchestration to create dataset, run each model in-turn and collect results.
# To keep the top of the notebook compact, the full orchestration is below and will be executed when you run this script as a notebook.
# The orchestration code uses the functions above and writes plots and a summary DataFrame.

# End of file.  
# Run the notebook cell-by-cell in Jupyter. The following orchestration is provided in the canvas and executed in the notebook environment.


LSTM

In [10]:
# --- Dataset Setup (must be run before training any model) ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("IMU.csv")   # <-- update path if needed
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# Apply your preprocessing & Kalman filter
from preprocessing import preprocess_data
from kalman_filter import apply_kalman_filter

# Kalman filter GPS
filtered_positions = apply_kalman_filter(df)
df['kf_latitude'] = filtered_positions[:,0]
df['kf_longitude'] = filtered_positions[:,1]

# Select sensor columns
sensor_cols = ['ax','ay','az','wx','wy','wz','Bx','By','Bz','speed']
X = df[sensor_cols].values
y = df[['kf_latitude','kf_longitude']].values

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Preprocess
X_train_prep, X_test_prep, scaler = preprocess_data(X_train, X_test, apply_smoothing=True)

print("X_train_prep shape:", X_train_prep.shape)
print("X_test_prep shape:", X_test_prep.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


Dataset shape: (10304, 14)
Columns: ['time', 'ax', 'ay', 'az', 'wx', 'wy', 'wz', 'Bx', 'By', 'Bz', 'latitude', 'longitude', 'altitude', 'speed']
X_train_prep shape: (8243, 10)
X_test_prep shape: (2061, 10)
y_train shape: (8243, 2)
y_test shape: (2061, 2)


In [11]:
# --- LSTM Experiment: Imports & Config --- CELL1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.metrics import r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Config
SEQ_LEN = 50
EPOCHS = 50
BATCH_SIZE = 64


In [12]:
# --- Sequence Creation Function --- CELL2
def create_sequences(X, y, seq_len=SEQ_LEN):
    n_samples = X.shape[0] - seq_len
    Xs = np.zeros((n_samples, seq_len, X.shape[1]))
    ys = np.zeros((n_samples, 2))
    for i in range(n_samples):
        Xs[i] = X[i:i+seq_len]
        ys[i] = y[i+seq_len]
    return Xs, ys

# Create sequences for train and test
X_train_seq, y_train_seq = create_sequences(X_train_prep, y_train, seq_len=SEQ_LEN)
X_test_seq, y_test_seq = create_sequences(X_test_prep, y_test, seq_len=SEQ_LEN)

print("Train sequences:", X_train_seq.shape, y_train_seq.shape)
print("Test sequences:", X_test_seq.shape, y_test_seq.shape)


Train sequences: (8193, 50, 10) (8193, 2)
Test sequences: (2011, 50, 10) (2011, 2)


In [13]:
# --- LSTM Model Definition ---
def build_lstm(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = LSTM(128, return_sequences=False)(inp)
    x = Dense(64, activation='relu')(x)
    out = Dense(2, activation='linear')(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

# Build model
model = build_lstm(X_train_seq.shape[1], X_train_seq.shape[2])

# Train model
es = EarlyStopping(patience=5, restore_best_weights=True)
t0 = time.perf_counter()
history = model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es],
    verbose=1
)
training_time = time.perf_counter() - t0


Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - loss: 2293.3474 - val_loss: 20.2892
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 2.6952 - val_loss: 1.1460
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - loss: 5.3968e-04 - val_loss: 1.1459
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - loss: 4.3470e-04 - val_loss: 1.1459
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 3.8126e-04 - val_loss: 1.1459
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - loss: 3.4355e-04 - val_loss: 1.1459
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - loss: 3.5839e-04 - val_loss: 1.1460
Epoch 8/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 3.5150e-04 - val_loss: 1.1460
Epoch 9/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - loss: 3.2006e-04 - val_loss: 1.1460
Epoch 10/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 2.7967e-04 - val_loss: 1.1460


In [14]:
# --- Predictions ---
t1 = time.perf_counter()
y_pred_seq = model.predict(X_test_seq)
inference_time = time.perf_counter() - t1

print("Predictions shape:", y_pred_seq.shape)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Predictions shape: (2011, 2)


In [103]:
# --- Metrics ---
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def compute_metrics_latlon(y_true, y_pred):
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    rmse = np.sqrt(np.mean(dists**2))
    mae = np.mean(np.abs(dists))
    r2_lat = r2_score(y_true[:,0], y_pred[:,0])
    r2_lon = r2_score(y_true[:,1], y_pred[:,1])
    r2 = (r2_lat + r2_lon) / 2.0
    avg_distance_error = np.mean(dists)

    def compute_trajectory_length_km(latitudes, longitudes):
        d = 0.0
        for i in range(1, len(latitudes)):
            d += haversine_dist(latitudes[i-1], longitudes[i-1], latitudes[i], longitudes[i])
        return d/1000.0
    traj_km = compute_trajectory_length_km(y_true[:,0], y_true[:,1])
    drift_per_km = (avg_distance_error / 1000.0) / (traj_km if traj_km>0 else np.nan)

    return {
        'RMSE (m)': rmse,
        'MAE (m)': mae,
        'R2': r2,
        'Avg Distance Error (m)': avg_distance_error,
        'Drift per km (m/km)': drift_per_km,
        'Training Time (s)': training_time,
        'Inference Time (s)': inference_time
    }

metrics_lstm = compute_metrics_latlon(y_test_seq, y_pred_seq)
metrics_lstm


{'RMSE (m)': 106545.4968596427,
 'MAE (m)': 4931.644893317383,
 'R2': -328364915.7948525,
 'Avg Distance Error (m)': 4931.644893317383,
 'Drift per km (m/km)': 80.37304073231284,
 'Training Time (s)': 59.323991500015836,
 'Inference Time (s)': 1.302601000003051}

In [106]:
# --- Show first 10 predictions ---
n = 10
df_preds = pd.DataFrame({
    "True_Lat": y_test_seq[:n,0],
    "True_Lon": y_test_seq[:n,1],
    "Pred_Lat": y_pred_seq[:n,0],
    "Pred_Lon": y_pred_seq[:n,1],
})
df_preds



,True_Lat,True_Lon,Pred_Lat,Pred_Lon
0,12.861877,77.664635,12.863280,77.673523
1,12.861877,77.664635,12.863280,77.673485
2,12.861877,77.664635,12.863297,77.673523
3,12.861877,77.664635,12.863311,77.673561
4,12.861877,77.664635,12.863301,77.673523
5,12.861877,77.664635,12.863281,77.673439
6,12.861877,77.664635,12.863273,77.673386
7,12.861877,77.664635,12.863265,77.673340
8,12.861877,77.664635,12.863269,77.673340
9,12.861877,77.664635,12.863281,77.673386


In [18]:
# --- Generate Dummy Sensor Data ---
def generate_dummy_sensor_data(df, n_samples=500):
    sensor_cols = ['ax','ay','az','wx','wy','wz','Bx','By','Bz','speed']
    dummy = pd.DataFrame()
    for col in sensor_cols:
        mu = df[col].mean()
        sigma = df[col].std()
        dummy[col] = np.random.normal(mu, sigma, n_samples)
    return dummy

dummy_df = generate_dummy_sensor_data(df, n_samples=500)
print("Dummy sensor data shape:", dummy_df.shape)
dummy_df.head()


Dummy sensor data shape: (500, 10)


,ax,ay,az,wx,wy,wz,Bx,By,Bz,speed
0,-1.160455,-0.228080,1.442801,0.320190,-0.145156,0.516786,-30.9249,29.025,-25.3177,0.130799
1,0.677195,-1.001082,-0.319133,-0.323365,0.051561,0.209995,-30.9249,29.025,-25.3177,0.466717
2,0.499264,-0.603857,-0.046761,0.213488,-0.305474,1.589007,-30.9249,29.025,-25.3177,0.176241
3,-0.058479,0.104060,-0.296504,0.062586,-0.077358,-0.439179,-30.9249,29.025,-25.3177,-1.410521
4,-1.257791,0.187634,-0.910177,0.181791,0.082172,0.202415,-30.9249,29.025,-25.3177,-0.751771


In [19]:
# --- Preprocess Dummy Data ---
dummy_scaled = scaler.transform(dummy_df.values)

# --- Create Sequences ---
dummy_seq, _ = create_sequences(dummy_scaled, np.zeros((dummy_scaled.shape[0], 2)), seq_len=SEQ_LEN)
print("Dummy sequence shape:", dummy_seq.shape)


Dummy sequence shape: (450, 50, 10)


In [20]:
# --- Predict GPS from Dummy IMU ---
dummy_pred = model.predict(dummy_seq)
print("Predicted GPS shape:", dummy_pred.shape)

# Show first 10 predictions
df_dummy_pred = pd.DataFrame({
    "Pred_Lat": dummy_pred[:10,0],
    "Pred_Lon": dummy_pred[:10,1]
})
df_dummy_pred


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Predicted GPS shape: (450, 2)


,Pred_Lat,Pred_Lon
0,12.865144,77.640610
1,12.866679,77.688301
2,12.846932,77.636017
3,12.876655,77.685608
4,12.860229,77.657822
5,12.861180,77.659233
6,12.856456,77.642303
7,12.863578,77.672546
8,12.863342,77.670837
9,12.857615,77.647583


In [21]:
# --- Compare Dummy Predictions to Real GPS (Optional Research Simulation) ---
real_subset = df[['kf_latitude','kf_longitude']].iloc[:len(dummy_pred)].values

compare_df = pd.DataFrame({
    "Real_Lat": real_subset[:10,0],
    "Real_Lon": real_subset[:10,1],
    "Pred_Lat": dummy_pred[:10,0],
    "Pred_Lon": dummy_pred[:10,1]
})
compare_df


,Real_Lat,Real_Lon,Pred_Lat,Pred_Lon
0,12.797868,77.278510,12.865144,77.640610
1,12.980259,78.379853,12.866679,77.688301
2,12.942670,78.152878,12.846932,77.636017
3,12.919576,78.013428,12.876655,77.685608
4,12.902972,77.913166,12.860229,77.657822
5,12.889839,77.833866,12.861180,77.659233
6,12.879427,77.770993,12.856456,77.642303
7,12.871672,77.724167,12.863578,77.672546
8,12.866429,77.692504,12.863342,77.670837
9,12.863277,77.673474,12.857615,77.647583


BiLSTM

In [23]:
# --- Sequence Creation Function ---
def create_sequences(X, y, seq_len=SEQ_LEN):
    n_samples = X.shape[0] - seq_len
    Xs = np.zeros((n_samples, seq_len, X.shape[1]))
    ys = np.zeros((n_samples, 2))
    for i in range(n_samples):
        Xs[i] = X[i:i+seq_len]
        ys[i] = y[i+seq_len]
    return Xs, ys

# Create sequences for train and test
X_train_seq, y_train_seq = create_sequences(X_train_prep, y_train, seq_len=SEQ_LEN)
X_test_seq, y_test_seq = create_sequences(X_test_prep, y_test, seq_len=SEQ_LEN)

print("Train sequences:", X_train_seq.shape, y_train_seq.shape)
print("Test sequences:", X_test_seq.shape, y_test_seq.shape)


# --- BiLSTM Model Definition ---
def build_bilstm(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = Bidirectional(LSTM(96, return_sequences=False))(inp)
    x = Dense(64, activation='relu')(x)
    out = Dense(2, activation='linear')(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

# Build model
bilstm_model = build_bilstm(X_train_seq.shape[1], X_train_seq.shape[2])

# Train model
es = EarlyStopping(patience=5, restore_best_weights=True)
t0 = time.perf_counter()
history_bilstm = bilstm_model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es],
    verbose=1
)
training_time_bilstm = time.perf_counter() - t0


Train sequences: (8193, 50, 10) (8193, 2)
Test sequences: (2011, 50, 10) (2011, 2)
Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 2041.6476 - val_loss: 0.7963
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 5.6799 - val_loss: 0.7931
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 1.5326 - val_loss: 0.7457
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.6358 - val_loss: 0.9171
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.1665 - val_loss: 0.9226
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0853 - val_loss: 0.9186
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0528 - val_loss: 0.9142
Epoch 8/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0256 - val_loss: 0.9151


In [24]:
# --- Predictions ---
t1 = time.perf_counter()
y_pred_bilstm = bilstm_model.predict(X_test_seq)
inference_time_bilstm = time.perf_counter() - t1

print("Predictions shape:", y_pred_bilstm.shape)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Predictions shape: (2011, 2)


In [25]:
# --- Metrics ---
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def compute_metrics_latlon(y_true, y_pred, training_time, inference_time):
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    rmse = np.sqrt(np.mean(dists**2))
    mae = np.mean(np.abs(dists))
    r2_lat = r2_score(y_true[:,0], y_pred[:,0])
    r2_lon = r2_score(y_true[:,1], y_pred[:,1])
    r2 = (r2_lat + r2_lon) / 2.0
    avg_distance_error = np.mean(dists)

    def compute_trajectory_length_km(latitudes, longitudes):
        d = 0.0
        for i in range(1, len(latitudes)):
            d += haversine_dist(latitudes[i-1], longitudes[i-1], latitudes[i], longitudes[i])
        return d/1000.0
    traj_km = compute_trajectory_length_km(y_true[:,0], y_true[:,1])
    drift_per_km = (avg_distance_error / 1000.0) / (traj_km if traj_km>0 else np.nan)

    return {
        'RMSE (m)': rmse,
        'MAE (m)': mae,
        'R2': r2,
        'Avg Distance Error (m)': avg_distance_error,
        'Drift per km (m/km)': drift_per_km,
        'Training Time (s)': training_time,
        'Inference Time (s)': inference_time
    }

metrics_bilstm = compute_metrics_latlon(y_test_seq, y_pred_bilstm, training_time_bilstm, inference_time_bilstm)
metrics_bilstm


{'RMSE (m)': 191499.6215708514,
 'MAE (m)': 62280.20228858848,
 'R2': -1073632869.1602051,
 'Avg Distance Error (m)': 62280.20228858848,
 'Drift per km (m/km)': 1015.0060159725411,
 'Training Time (s)': 29.972453100024723,
 'Inference Time (s)': 1.2150336999911815}

In [26]:
# --- Show first 10 predictions ---
n = 10
df_preds_bilstm = pd.DataFrame({
    "True_Lat": y_test_seq[:n,0],
    "True_Lon": y_test_seq[:n,1],
    "Pred_Lat": y_pred_bilstm[:n,0],
    "Pred_Lon": y_pred_bilstm[:n,1],
})
df_preds_bilstm


,True_Lat,True_Lon,Pred_Lat,Pred_Lon
0,12.861877,77.664635,10.645909,53.734543
1,12.861877,77.664635,11.256668,58.995911
2,12.861877,77.664635,12.852773,77.814438
3,12.861877,77.664635,12.852775,77.815689
4,12.861877,77.664635,12.853385,77.822006
5,12.861877,77.664635,12.854795,77.833771
6,12.861877,77.664635,12.856912,77.849350
7,12.861877,77.664635,12.859162,77.865982
8,12.861877,77.664635,12.861217,77.880615
9,12.861877,77.664635,12.862825,77.892120


In [28]:
def generate_dummy_sensor_data(df, n_samples=500):
    sensor_cols = ['ax','ay','az','wx','wy','wz','Bx','By','Bz','speed']
    dummy = pd.DataFrame()
    for col in sensor_cols:
        mu = df[col].mean()
        sigma = df[col].std()
        dummy[col] = np.random.normal(mu, sigma, n_samples)
    return dummy

dummy_df = generate_dummy_sensor_data(df, n_samples=500)
print("Dummy sensor data shape:", dummy_df.shape)
dummy_df.head()


Dummy sensor data shape: (500, 10)


,ax,ay,az,wx,wy,wz,Bx,By,Bz,speed
0,-0.435452,-0.636635,-2.544119,-0.373808,-0.093785,0.619349,-30.9249,29.025,-25.3177,-0.869616
1,-0.235346,-1.191128,-1.870547,-0.416716,-0.066396,0.670542,-30.9249,29.025,-25.3177,1.316417
2,0.383915,-0.589084,-0.347886,-0.064270,-0.057164,0.062174,-30.9249,29.025,-25.3177,0.594820
3,-0.662219,0.451768,-1.085385,-0.332374,0.130390,-0.105234,-30.9249,29.025,-25.3177,0.964136
4,-0.697246,0.939834,-0.075627,0.196359,-0.148740,0.504286,-30.9249,29.025,-25.3177,-0.344417


In [29]:
dummy_scaled = scaler.transform(dummy_df.values)
dummy_seq, _ = create_sequences(dummy_scaled, np.zeros((dummy_scaled.shape[0], 2)), seq_len=SEQ_LEN)
print("Dummy sequence shape:", dummy_seq.shape)


Dummy sequence shape: (450, 50, 10)


In [30]:
dummy_pred_bilstm = bilstm_model.predict(dummy_seq)
print("Predicted GPS shape:", dummy_pred_bilstm.shape)

df_dummy_pred_bilstm = pd.DataFrame({
    "Pred_Lat": dummy_pred_bilstm[:10,0],
    "Pred_Lon": dummy_pred_bilstm[:10,1]
})
df_dummy_pred_bilstm


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Predicted GPS shape: (450, 2)


,Pred_Lat,Pred_Lon
0,12.946026,78.165977
1,12.609485,75.615753
2,12.850123,77.678680
3,12.849887,77.856003
4,12.607202,75.585350
5,12.576129,75.460083
6,12.658645,75.457581
7,12.614738,75.599350
8,12.853844,77.773087
9,12.884387,77.811905


In [31]:
real_subset = df[['kf_latitude','kf_longitude']].iloc[:len(dummy_pred_bilstm)].values

compare_df_bilstm = pd.DataFrame({
    "Real_Lat": real_subset[:10,0],
    "Real_Lon": real_subset[:10,1],
    "Pred_Lat": dummy_pred_bilstm[:10,0],
    "Pred_Lon": dummy_pred_bilstm[:10,1]
})
compare_df_bilstm


,Real_Lat,Real_Lon,Pred_Lat,Pred_Lon
0,12.797868,77.278510,12.946026,78.165977
1,12.980259,78.379853,12.609485,75.615753
2,12.942670,78.152878,12.850123,77.678680
3,12.919576,78.013428,12.849887,77.856003
4,12.902972,77.913166,12.607202,75.585350
5,12.889839,77.833866,12.576129,75.460083
6,12.879427,77.770993,12.658645,75.457581
7,12.871672,77.724167,12.614738,75.599350
8,12.866429,77.692504,12.853844,77.773087
9,12.863277,77.673474,12.884387,77.811905


CNN + LSTM

In [33]:
# --- Sequence Creation Function ---
def create_sequences(X, y, seq_len=SEQ_LEN):
    n_samples = X.shape[0] - seq_len
    Xs = np.zeros((n_samples, seq_len, X.shape[1]))
    ys = np.zeros((n_samples, 2))
    for i in range(n_samples):
        Xs[i] = X[i:i+seq_len]
        ys[i] = y[i+seq_len]
    return Xs, ys

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_prep, y_train, seq_len=SEQ_LEN)
X_test_seq, y_test_seq = create_sequences(X_test_prep, y_test, seq_len=SEQ_LEN)

print("Train sequences:", X_train_seq.shape, y_train_seq.shape)
print("Test sequences:", X_test_seq.shape, y_test_seq.shape)


Train sequences: (8193, 50, 10) (8193, 2)
Test sequences: (2011, 50, 10) (2011, 2)


In [34]:
# --- CNN+LSTM Model Definition ---
def build_cnn_lstm(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = Conv1D(64, 3, padding='same', activation='relu')(inp)
    x = MaxPooling1D(pool_size=2)(x)
    x = LSTM(64)(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(2)(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

# Build model
cnn_lstm_model = build_cnn_lstm(X_train_seq.shape[1], X_train_seq.shape[2])

# Train model
es = EarlyStopping(patience=5, restore_best_weights=True)
t0 = time.perf_counter()
history_cnn_lstm = cnn_lstm_model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es],
    verbose=1
)
training_time_cnn_lstm = time.perf_counter() - t0


Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 2408.3345 - val_loss: 458.5397
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 215.3308 - val_loss: 0.3253
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0647 - val_loss: 0.0050
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0036 - val_loss: 0.0051
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0035 - val_loss: 0.0051
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0035 - val_loss: 0.0046
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0035 - val_loss: 0.0046
Epoch 8/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0036 - val_loss: 0.0047
Epoch 9/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0035 - val_loss: 0.0049
Epoch 10/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0034 - val_loss: 0.0047
Epoch 11/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0034 - val_loss: 0.0045
Epoch 12/50
116/116 ━━━━━━━━━━━━━━

In [35]:
# --- Predictions ---
t1 = time.perf_counter()
y_pred_cnn_lstm = cnn_lstm_model.predict(X_test_seq)
inference_time_cnn_lstm = time.perf_counter() - t1

print("Predictions shape:", y_pred_cnn_lstm.shape)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Predictions shape: (2011, 2)


In [36]:
# --- Metrics ---
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def compute_metrics_latlon(y_true, y_pred, training_time, inference_time):
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    rmse = np.sqrt(np.mean(dists**2))
    mae = np.mean(np.abs(dists))
    r2_lat = r2_score(y_true[:,0], y_pred[:,0])
    r2_lon = r2_score(y_true[:,1], y_pred[:,1])
    r2 = (r2_lat + r2_lon) / 2.0
    avg_distance_error = np.mean(dists)

    def compute_trajectory_length_km(latitudes, longitudes):
        d = 0.0
        for i in range(1, len(latitudes)):
            d += haversine_dist(latitudes[i-1], longitudes[i-1], latitudes[i], longitudes[i])
        return d/1000.0
    traj_km = compute_trajectory_length_km(y_true[:,0], y_true[:,1])
    drift_per_km = (avg_distance_error / 1000.0) / (traj_km if traj_km>0 else np.nan)

    return {
        'RMSE (m)': rmse,
        'MAE (m)': mae,
        'R2': r2,
        'Avg Distance Error (m)': avg_distance_error,
        'Drift per km (m/km)': drift_per_km,
        'Training Time (s)': training_time,
        'Inference Time (s)': inference_time
    }

metrics_cnn_lstm = compute_metrics_latlon(y_test_seq, y_pred_cnn_lstm, training_time_cnn_lstm, inference_time_cnn_lstm)
metrics_cnn_lstm


{'RMSE (m)': 1310.8295840461776,
 'MAE (m)': 1122.0165217375547,
 'R2': -52594.12624145855,
 'Avg Distance Error (m)': 1122.0165217375547,
 'Drift per km (m/km)': 18.285963720976454,
 'Training Time (s)': 70.42916439997498,
 'Inference Time (s)': 0.9010910000069998}

In [37]:
n = 10
df_preds_cnn_lstm = pd.DataFrame({
    "True_Lat": y_test_seq[:n,0],
    "True_Lon": y_test_seq[:n,1],
    "Pred_Lat": y_pred_cnn_lstm[:n,0],
    "Pred_Lon": y_pred_cnn_lstm[:n,1],
})
df_preds_cnn_lstm


,True_Lat,True_Lon,Pred_Lat,Pred_Lon
0,12.861877,77.664635,12.868470,77.657318
1,12.861877,77.664635,12.868569,77.659096
2,12.861877,77.664635,12.868570,77.660278
3,12.861877,77.664635,12.868473,77.660767
4,12.861877,77.664635,12.868277,77.660164
5,12.861877,77.664635,12.867944,77.658348
6,12.861877,77.664635,12.867568,77.655830
7,12.861877,77.664635,12.867370,77.653839
8,12.861877,77.664635,12.867379,77.652946
9,12.861877,77.664635,12.867473,77.653175


In [40]:
def generate_dummy_sensor_data(df, n_samples=500):
    sensor_cols = ['ax','ay','az','wx','wy','wz','Bx','By','Bz','speed']
    dummy = pd.DataFrame()
    for col in sensor_cols:
        mu = df[col].mean()
        sigma = df[col].std()
        dummy[col] = np.random.normal(mu, sigma, n_samples)
    return dummy

dummy_df = generate_dummy_sensor_data(df, n_samples=500)
print("Dummy sensor data shape:", dummy_df.shape)
dummy_df.head()


dummy_scaled = scaler.transform(dummy_df.values)
dummy_seq, _ = create_sequences(dummy_scaled, np.zeros((dummy_scaled.shape[0], 2)), seq_len=SEQ_LEN)
print("Dummy sequence shape:", dummy_seq.shape)


dummy_pred_cnn_lstm = cnn_lstm_model.predict(dummy_seq)
print("Predicted GPS shape:", dummy_pred_cnn_lstm.shape)

df_dummy_pred_cnn_lstm = pd.DataFrame({
    "Pred_Lat": dummy_pred_cnn_lstm[:10,0],
    "Pred_Lon": dummy_pred_cnn_lstm[:10,1]
})
df_dummy_pred_cnn_lstm


Dummy sensor data shape: (500, 10)
Dummy sequence shape: (450, 50, 10)
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Predicted GPS shape: (450, 2)


,Pred_Lat,Pred_Lon
0,12.850790,77.653824
1,12.859399,77.684380
2,12.857165,77.676247
3,12.857635,77.688049
4,12.849034,77.643501
5,12.852536,77.655182
6,12.851193,77.629883
7,12.857131,77.672165
8,12.848262,77.643349
9,12.852971,77.657440


In [41]:
real_subset = df[['kf_latitude','kf_longitude']].iloc[:len(dummy_pred_cnn_lstm)].values

compare_df_cnn_lstm = pd.DataFrame({
    "Real_Lat": real_subset[:10,0],
    "Real_Lon": real_subset[:10,1],
    "Pred_Lat": dummy_pred_cnn_lstm[:10,0],
    "Pred_Lon": dummy_pred_cnn_lstm[:10,1]
})
compare_df_cnn_lstm


,Real_Lat,Real_Lon,Pred_Lat,Pred_Lon
0,12.797868,77.278510,12.850790,77.653824
1,12.980259,78.379853,12.859399,77.684380
2,12.942670,78.152878,12.857165,77.676247
3,12.919576,78.013428,12.857635,77.688049
4,12.902972,77.913166,12.849034,77.643501
5,12.889839,77.833866,12.852536,77.655182
6,12.879427,77.770993,12.851193,77.629883
7,12.871672,77.724167,12.857131,77.672165
8,12.866429,77.692504,12.848262,77.643349
9,12.863277,77.673474,12.852971,77.657440


TRANSFORMER

In [48]:
# --- Transformer Model Definition ---
def build_transformer(seq_len, n_feats, head_size=64, num_heads=4, ff_dim=128):
    inp = Input(shape=(seq_len, n_feats))
    x = Dense(head_size)(inp)
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    x = LayerNormalization()(attn + x)

    # Feed Forward block
    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(head_size)(ff)
    x = LayerNormalization()(ff + x)

    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(2)(x)
    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

# Build model
transformer_model = build_transformer(X_train_seq.shape[1], X_train_seq.shape[2])

# Train model
es = EarlyStopping(patience=5, restore_best_weights=True)
t0 = time.perf_counter()
history_transformer = transformer_model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es],
    verbose=1
)
training_time_transformer = time.perf_counter() - t0


Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 1966.4097 - val_loss: 1.0137
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.4252 - val_loss: 0.6979
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0054 - val_loss: 0.7013
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0051 - val_loss: 0.7050
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0045 - val_loss: 0.7086
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0039 - val_loss: 0.7134
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0037 - val_loss: 0.7180


In [49]:
t1 = time.perf_counter()
y_pred_transformer = transformer_model.predict(X_test_seq)
inference_time_transformer = time.perf_counter() - t1

print("Predictions shape:", y_pred_transformer.shape)


def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def compute_metrics_latlon(y_true, y_pred, training_time, inference_time):
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    rmse = np.sqrt(np.mean(dists**2))
    mae = np.mean(np.abs(dists))
    r2_lat = r2_score(y_true[:,0], y_pred[:,0])
    r2_lon = r2_score(y_true[:,1], y_pred[:,1])
    r2 = (r2_lat + r2_lon) / 2.0
    avg_distance_error = np.mean(dists)

    def compute_trajectory_length_km(latitudes, longitudes):
        d = 0.0
        for i in range(1, len(latitudes)):
            d += haversine_dist(latitudes[i-1], longitudes[i-1], latitudes[i], longitudes[i])
        return d/1000.0
    traj_km = compute_trajectory_length_km(y_true[:,0], y_true[:,1])
    drift_per_km = (avg_distance_error / 1000.0) / (traj_km if traj_km>0 else np.nan)

    return {
        'RMSE (m)': rmse,
        'MAE (m)': mae,
        'R2': r2,
        'Avg Distance Error (m)': avg_distance_error,
        'Drift per km (m/km)': drift_per_km,
        'Training Time (s)': training_time,
        'Inference Time (s)': inference_time
    }

metrics_transformer = compute_metrics_latlon(y_test_seq, y_pred_transformer, training_time_transformer, inference_time_transformer)
metrics_transformer


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Predictions shape: (2011, 2)


{'RMSE (m)': 142088.33585697002,
 'MAE (m)': 18232.522115968335,
 'R2': -585835827.2741318,
 'Avg Distance Error (m)': 18232.522115968335,
 'Drift per km (m/km)': 297.14289539889825,
 'Training Time (s)': 15.406971500022337,
 'Inference Time (s)': 0.7493307000258937}

In [111]:
n = 1000
df_preds_transformer = pd.DataFrame({
    "True_Lat": y_test_seq[:n,0],
    "True_Lon": y_test_seq[:n,1],
    "Pred_Lat": y_pred_transformer[:n,0],
    "Pred_Lon": y_pred_transformer[:n,1],
})
df_preds_transformer


,True_Lat,True_Lon,Pred_Lat,Pred_Lon
0,12.861877,77.664635,8.964962,50.899399
1,12.861877,77.664635,5.909423,32.820408
2,12.861877,77.664635,12.898383,77.742134
3,12.861877,77.664635,12.898170,77.742027
4,12.861877,77.664635,12.897894,77.741760
...,...,...,...,...
995,12.861889,77.664671,12.877929,77.666039
996,12.861889,77.664671,12.875927,77.660362
997,12.861889,77.664671,12.873409,77.651222
998,12.861889,77.664671,12.869089,77.630882


In [54]:
def generate_dummy_sensor_data(df, n_samples=500):
    sensor_cols = ['ax','ay','az','wx','wy','wz','Bx','By','Bz','speed']
    dummy = pd.DataFrame()
    for col in sensor_cols:
        mu = df[col].mean()
        sigma = df[col].std()
        dummy[col] = np.random.normal(mu, sigma, n_samples)
    return dummy

dummy_df = generate_dummy_sensor_data(df, n_samples=500)
print("Dummy sensor data shape:", dummy_df.shape)
dummy_df.head()


Dummy sensor data shape: (500, 10)


,ax,ay,az,wx,wy,wz,Bx,By,Bz,speed
0,-0.670058,-1.308907,-0.284211,0.142326,-0.418151,0.067333,-30.9249,29.025,-25.3177,-0.970869
1,-0.280394,-0.014932,2.082430,0.439432,0.344040,0.006121,-30.9249,29.025,-25.3177,0.493256
2,0.475547,-0.693535,-0.451551,-0.427294,0.299358,0.102002,-30.9249,29.025,-25.3177,0.177972
3,-0.180238,-0.778166,-0.976868,0.075904,0.312075,-0.030986,-30.9249,29.025,-25.3177,0.434026
4,-0.450356,-0.299529,-0.087009,-0.277613,-0.128942,-0.537324,-30.9249,29.025,-25.3177,-0.028323


In [55]:
dummy_scaled = scaler.transform(dummy_df.values)
dummy_seq, _ = create_sequences(dummy_scaled, np.zeros((dummy_scaled.shape[0], 2)), seq_len=SEQ_LEN)
print("Dummy sequence shape:", dummy_seq.shape)


Dummy sequence shape: (450, 50, 10)


In [56]:
dummy_pred_transformer = transformer_model.predict(dummy_seq)
print("Predicted GPS shape:", dummy_pred_transformer.shape)

df_dummy_pred_transformer = pd.DataFrame({
    "Pred_Lat": dummy_pred_transformer[:10,0],
    "Pred_Lon": dummy_pred_transformer[:10,1]
})
df_dummy_pred_transformer


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Predicted GPS shape: (450, 2)


,Pred_Lat,Pred_Lon
0,12.828369,77.346573
1,12.827088,77.354088
2,12.828128,77.369637
3,12.828266,77.370689
4,12.827586,77.384209
5,12.832289,77.379402
6,12.832212,77.386314
7,12.832926,77.397041
8,12.830916,77.385201
9,12.826620,77.375557


In [57]:
real_subset = df[['kf_latitude','kf_longitude']].iloc[:len(dummy_pred_transformer)].values

compare_df_transformer = pd.DataFrame({
    "Real_Lat": real_subset[:10,0],
    "Real_Lon": real_subset[:10,1],
    "Pred_Lat": dummy_pred_transformer[:10,0],
    "Pred_Lon": dummy_pred_transformer[:10,1]
})
compare_df_transformer


,Real_Lat,Real_Lon,Pred_Lat,Pred_Lon
0,12.797868,77.278510,12.828369,77.346573
1,12.980259,78.379853,12.827088,77.354088
2,12.942670,78.152878,12.828128,77.369637
3,12.919576,78.013428,12.828266,77.370689
4,12.902972,77.913166,12.827586,77.384209
5,12.889839,77.833866,12.832289,77.379402
6,12.879427,77.770993,12.832212,77.386314
7,12.871672,77.724167,12.832926,77.397041
8,12.866429,77.692504,12.830916,77.385201
9,12.863277,77.673474,12.826620,77.375557


ResNet1D

In [59]:
# --- Sequence Creation ---
def create_sequences(X, y, seq_len=SEQ_LEN):
    n_samples = X.shape[0] - seq_len
    Xs = np.zeros((n_samples, seq_len, X.shape[1]))
    ys = np.zeros((n_samples, 2))
    for i in range(n_samples):
        Xs[i] = X[i:i+seq_len]
        ys[i] = y[i+seq_len]
    return Xs, ys

X_train_seq, y_train_seq = create_sequences(X_train_prep, y_train, seq_len=SEQ_LEN)
X_test_seq, y_test_seq = create_sequences(X_test_prep, y_test, seq_len=SEQ_LEN)

print("Train sequences:", X_train_seq.shape, y_train_seq.shape)
print("Test sequences:", X_test_seq.shape, y_test_seq.shape)


Train sequences: (8193, 50, 10) (8193, 2)
Test sequences: (2011, 50, 10) (2011, 2)


In [60]:
# --- ResNet1D Model Definition ---
def residual_block(x, filters, kernel_size=3):
    y = Conv1D(filters, kernel_size, padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Conv1D(filters, kernel_size, padding='same')(y)
    y = BatchNormalization()(y)
    y = Add()([x, y])
    y = Activation('relu')(y)
    return y

def build_resnet1d(seq_len, n_feats):
    inp = Input(shape=(seq_len, n_feats))
    x = Conv1D(64, 3, padding='same')(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Add a couple of residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)

    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(2)(x)

    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

# Build model
resnet_model = build_resnet1d(X_train_seq.shape[1], X_train_seq.shape[2])

# Train model
es = EarlyStopping(patience=5, restore_best_weights=True)
t0 = time.perf_counter()
history_resnet = resnet_model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es],
    verbose=1
)
training_time_resnet = time.perf_counter() - t0


Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 2101.2893 - val_loss: 332.8287
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6.7878 - val_loss: 145.0201
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.3794 - val_loss: 49.1089
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.3701 - val_loss: 9.7050
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.5472 - val_loss: 3.0686
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.4514 - val_loss: 0.4540
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.2714 - val_loss: 0.7343
Epoch 8/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3818 - val_loss: 0.6888
Epoch 9/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3177 - val_loss: 0.5985
Epoch 10/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.1894 - val_loss: 0.0834
Epoch 11/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.2554 - val_loss: 0.8014
Epoch 12/50
116/116 ━━━━━━━━━━━━━

In [61]:
t1 = time.perf_counter()
y_pred_resnet = resnet_model.predict(X_test_seq)
inference_time_resnet = time.perf_counter() - t1

print("Predictions shape:", y_pred_resnet.shape)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Predictions shape: (2011, 2)


In [62]:
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def compute_metrics_latlon(y_true, y_pred, training_time, inference_time):
    dists = haversine_dist(y_true[:,0], y_true[:,1], y_pred[:,0], y_pred[:,1])
    rmse = np.sqrt(np.mean(dists**2))
    mae = np.mean(np.abs(dists))
    r2_lat = r2_score(y_true[:,0], y_pred[:,0])
    r2_lon = r2_score(y_true[:,1], y_pred[:,1])
    r2 = (r2_lat + r2_lon) / 2.0
    avg_distance_error = np.mean(dists)

    def compute_trajectory_length_km(latitudes, longitudes):
        d = 0.0
        for i in range(1, len(latitudes)):
            d += haversine_dist(latitudes[i-1], longitudes[i-1], latitudes[i], longitudes[i])
        return d/1000.0
    traj_km = compute_trajectory_length_km(y_true[:,0], y_true[:,1])
    drift_per_km = (avg_distance_error / 1000.0) / (traj_km if traj_km>0 else np.nan)

    return {
        'RMSE (m)': rmse,
        'MAE (m)': mae,
        'R2': r2,
        'Avg Distance Error (m)': avg_distance_error,
        'Drift per km (m/km)': drift_per_km,
        'Training Time (s)': training_time,
        'Inference Time (s)': inference_time
    }

metrics_resnet = compute_metrics_latlon(y_test_seq, y_pred_resnet, training_time_resnet, inference_time_resnet)
metrics_resnet


{'RMSE (m)': 69233.84201633552,
 'MAE (m)': 56876.007184155256,
 'R2': -144208889.99497038,
 'Avg Distance Error (m)': 56876.007184155256,
 'Drift per km (m/km)': 926.9316305190093,
 'Training Time (s)': 31.687436699983664,
 'Inference Time (s)': 0.6591637000092305}

In [63]:
n = 10
df_preds_resnet = pd.DataFrame({
    "True_Lat": y_test_seq[:n,0],
    "True_Lon": y_test_seq[:n,1],
    "Pred_Lat": y_pred_resnet[:n,0],
    "Pred_Lon": y_pred_resnet[:n,1],
})
df_preds_resnet


,True_Lat,True_Lon,Pred_Lat,Pred_Lon
0,12.861877,77.664635,13.883780,82.603828
1,12.861877,77.664635,13.627173,79.803535
2,12.861877,77.664635,13.201361,77.757332
3,12.861877,77.664635,13.207400,77.745140
4,12.861877,77.664635,13.209126,77.702271
5,12.861877,77.664635,13.211705,77.660744
6,12.861877,77.664635,13.217700,77.638252
7,12.861877,77.664635,13.226438,77.633232
8,12.861877,77.664635,13.235525,77.638390
9,12.861877,77.664635,13.244840,77.663551


In [65]:
def generate_dummy_sensor_data(df, n_samples=500):
    sensor_cols = ['ax','ay','az','wx','wy','wz','Bx','By','Bz','speed']
    dummy = pd.DataFrame()
    for col in sensor_cols:
        mu = df[col].mean()
        sigma = df[col].std()
        dummy[col] = np.random.normal(mu, sigma, n_samples)
    return dummy

dummy_df = generate_dummy_sensor_data(df, n_samples=500)
print("Dummy sensor data shape:", dummy_df.shape)
dummy_df.head()


Dummy sensor data shape: (500, 10)


,ax,ay,az,wx,wy,wz,Bx,By,Bz,speed
0,0.093889,-0.249589,0.381000,-0.085394,0.289698,-0.386420,-30.9249,29.025,-25.3177,0.093287
1,-0.896146,0.183449,-0.107852,-0.313789,0.047272,-0.055806,-30.9249,29.025,-25.3177,1.017595
2,0.072022,0.171735,-1.044816,0.217971,-0.093568,0.884007,-30.9249,29.025,-25.3177,0.260043
3,0.821447,0.016651,-0.154150,0.784352,-0.046459,-0.055801,-30.9249,29.025,-25.3177,-0.235962
4,-0.985276,0.182233,0.783814,0.063974,-0.054079,0.295600,-30.9249,29.025,-25.3177,0.225385


In [66]:
dummy_scaled = scaler.transform(dummy_df.values)
dummy_seq, _ = create_sequences(dummy_scaled, np.zeros((dummy_scaled.shape[0], 2)), seq_len=SEQ_LEN)
print("Dummy sequence shape:", dummy_seq.shape)


Dummy sequence shape: (450, 50, 10)


In [67]:
dummy_pred_resnet = resnet_model.predict(dummy_seq)
print("Predicted GPS shape:", dummy_pred_resnet.shape)

df_dummy_pred_resnet = pd.DataFrame({
    "Pred_Lat": dummy_pred_resnet[:10,0],
    "Pred_Lon": dummy_pred_resnet[:10,1]
})
df_dummy_pred_resnet


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Predicted GPS shape: (450, 2)


,Pred_Lat,Pred_Lon
0,13.126073,76.114410
1,13.121699,76.286270
2,13.045911,76.160301
3,13.060356,76.324303
4,13.068346,76.281075
5,13.025187,76.364540
6,12.978835,76.088921
7,12.963710,76.230980
8,12.967242,75.996498
9,13.131426,76.727516


In [68]:
real_subset = df[['kf_latitude','kf_longitude']].iloc[:len(dummy_pred_resnet)].values

compare_df_resnet = pd.DataFrame({
    "Real_Lat": real_subset[:10,0],
    "Real_Lon": real_subset[:10,1],
    "Pred_Lat": dummy_pred_resnet[:10,0],
    "Pred_Lon": dummy_pred_resnet[:10,1]
})
compare_df_resnet


,Real_Lat,Real_Lon,Pred_Lat,Pred_Lon
0,12.797868,77.278510,13.126073,76.114410
1,12.980259,78.379853,13.121699,76.286270
2,12.942670,78.152878,13.045911,76.160301
3,12.919576,78.013428,13.060356,76.324303
4,12.902972,77.913166,13.068346,76.281075
5,12.889839,77.833866,13.025187,76.364540
6,12.879427,77.770993,12.978835,76.088921
7,12.871672,77.724167,12.963710,76.230980
8,12.866429,77.692504,12.967242,75.996498
9,12.863277,77.673474,13.131426,76.727516
